## Procedimentos
### Alternativa com frequência fixa
1. Selecionar intervalo de tempo do dado de entrada;
2. Criar um novo conjunto de dados igualmente espaçados (selecionando-se inicio, frequência e total);
3. Concatenar dados dos sensores e ordena-los por meio do timestamp;
4. 
----

In [61]:
# Packages and Libraries
import pandas as pd
import numpy as np

In [79]:
# Read database
# , parse_dates=["Time"], index_col="Time

df_wise_x = pd.read_csv('../Datasets/dataset_TPV/base_23042023_A/wise/x.csv')
df_wise_x = df_wise_x.rename(columns={'OAVelocity' : 'OAVelocity_x', 'Peakmg' : 'Peakmg_x', 'RMSmg' : 'RMSmg_x',
                                      'Kurtosis' : 'Kurtosis_x', 'CrestFactor' : 'CrestFactor_x', 'Skewness' : 'Skewness_x',
                                      'Deviation' : 'Deviation_x', 'Peak-to-Peak Displacement' : 'Peak-to-Peak Displacement_x'})

df_wise_y = pd.read_csv('../Datasets/dataset_TPV/base_23042023_A/wise/y.csv')
df_wise_y = df_wise_y.rename(columns={'OAVelocity' : 'OAVelocity_y', 'Peakmg' : 'Peakmg_y', 'RMSmg' : 'RMSmg_y',
                                      'Kurtosis' : 'Kurtosis_y', 'CrestFactor' : 'CrestFactor_y', 'Skewness' : 'Skewness_y',
                                      'Deviation' : 'Deviation_y', 'Peak-to-Peak Displacement' : 'Peak-to-Peak Displacement_y'})
df_wise_y = df_wise_y.drop('Time', axis='columns')

df_wise_z = pd.read_csv('../Datasets/dataset_TPV/base_23042023_A/wise/z.csv')
df_wise_z = df_wise_z.rename(columns={'OAVelocity' : 'OAVelocity_z', 'Peakmg' : 'Peakmg_z', 'RMSmg' : 'RMSmg_z',
                                      'Kurtosis' : 'Kurtosis_z', 'CrestFactor' : 'CrestFactor_z', 'Skewness' : 'Skewness_z',
                                      'Deviation' : 'Deviation_z', 'Peak-to-Peak Displacement' : 'Peak-to-Peak Displacement_z'})
df_wise_z = df_wise_z.drop('Time', axis='columns')

df_wise = pd.concat([df_wise_x, df_wise_y, df_wise_z], axis=1)
df_wise['Time'] = pd.to_datetime(df_wise['Time'])
df_wise.set_index('Time', inplace=True)

df_ite = pd.read_csv('../Datasets/dataset_TPV/base_23042023_A/ite/ite.csv', parse_dates=["Time"], index_col="Time")
# df_ite = df_ite.rename(columns={'Time':'Time_ite'})
# print(df_ite.info())

df_hexa = pd.read_csv('../Datasets/dataset_TPV/base_23042023_A/hex/hex.csv', parse_dates=["Time"], index_col="Time")
# df_hexa = df_hexa.rename(columns={'Time':'Time_hex'})
# df.sort_index()
# print(df_hexa.info())

# # Combine all variables into a single dataframe
# df_concat = pd.concat([df_hexa, df_ite, df_wise_x, df_wise_y, df_wise_z],axis=1)
df_concat = pd.concat([df_hexa, df_ite, df_wise])
df_concat.sort_index(inplace=True)
df_concat.to_csv('test_original.csv')
# print(df_concat.describe())

# print(df_concat.head())

In [80]:
# Fill NaN whit the previous value and drop NaN remain values
df_concat.fillna(method='pad', inplace=True)
df_concat.dropna(inplace=True)

# Remove duplicated timestamps keeping the last one
df = df_concat[::-1]
df = df.iloc[ np.unique( df.index.values, return_index = True )[1] ]

# Save new data to csv
df.to_csv('test_filled.csv')

In [90]:
# Resample data to determined frequency

freq = '600S'    # 1 minute
df_resampled = df.resample(freq, label='right', closed='right').agg('mean')
df_resampled.fillna(method='pad', inplace=True) # Preenche dados com os valores anteriores
df_resampled.to_csv('test_resampled_10min.csv')


In [89]:
df_resampled.isna().sum()

InletPressure                  0
OutletPressure                 0
OutletTemperature              0
InverterSpeed                  0
temperature                    0
frequency                      0
phaseA_voltage                 0
phaseA_current                 0
phaseA_pwr_factor              0
phaseA_active                  0
phaseA_reactive                0
phaseB_voltage                 0
phaseB_current                 0
phaseB_pwr_factor              0
phaseB_active                  0
phaseB_reactive                0
phaseC_voltage                 0
phaseC_current                 0
phaseC_pwr_factor              0
phaseC_active                  0
phaseC_reactive                0
OAVelocity_x                   0
Peakmg_x                       0
RMSmg_x                        0
Kurtosis_x                     0
CrestFactor_x                  0
Skewness_x                     0
Deviation_x                    0
Peak-to-Peak Displacement_x    0
OAVelocity_y                   0
Peakmg_y  

In [88]:
df_period = pd.read_csv('test_resampled.csv')
df_period['Time'] = pd.to_datetime(df_period['Time'])
df_period['Period'] = df_period['Time'].diff()

df1 = df_period['Period'].value_counts()
print(df_period['Period'].max())
print(df_period['Period'].mean())
print(df_period['Period'].min())
print(df_period['Period'].mode())

print(df1.sort_index())

0 days 00:01:00
0 days 00:01:00
0 days 00:01:00
0   0 days 00:01:00
Name: Period, dtype: timedelta64[ns]
0 days 00:01:00    81334
Name: Period, dtype: int64
